In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, random_split
from torchvision import datasets, transforms, models

import numpy as np
import random
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, classification_report
import seaborn as sns

In [ ]:
# Reproducibility
seed = 42
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed_all(seed)

torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

# Device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Parameters
lr = 1e-3
epochs = 25
batch = 128
num_classes = 10


In [ ]:
train_tfms = transforms.Compose([
    transforms.Resize(224),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize([0.485,0.456,0.406],[0.229,0.224,0.225])
])

test_tfms = transforms.Compose([
    transforms.Resize(224),
    transforms.ToTensor(),
    transforms.Normalize([0.485,0.456,0.406],[0.229,0.224,0.225])
])

full_train = datasets.CIFAR10(root='./data', train=True, transform=train_tfms, download=True)
test_set   = datasets.CIFAR10(root='./data', train=False, transform=test_tfms, download=True)

# Split train into train/val
train_len = int(0.8 * len(full_train))
val_len   = len(full_train) - train_len
train_set, val_set = random_split(full_train, [train_len, val_len])

train_loader = DataLoader(train_set, batch_size=batch, shuffle=True)
val_loader   = DataLoader(val_set, batch_size=batch, shuffle=False)
test_loader  = DataLoader(test_set, batch_size=batch, shuffle=False)


In [ ]:
# Load pretrained model
model = models.resnet18(weights=models.ResNet18_Weights.IMAGENET1K_V1)

# Freeze base layers
for p in model.parameters():
    p.requires_grad = False

# Unfreeze last block
for p in model.layer4.parameters():
    p.requires_grad = True

# Replace final FC layer
model.fc = nn.Linear(model.fc.in_features, num_classes)
model = model.to(device)


In [ ]:
criterion = nn.CrossEntropyLoss()
opt = optim.Adam(filter(lambda x: x.requires_grad, model.parameters()), lr=lr)
sched = torch.optim.lr_scheduler.StepLR(opt, step_size=10, gamma=0.1)

train_loss_hist, val_loss_hist = [], []
train_acc_hist,  val_acc_hist  = [], []



In [ ]:
for ep in range(epochs):
    model.train()
    running_loss, correct, total = 0.0, 0, 0

    for imgs, lbls in train_loader:
        imgs, lbls = imgs.to(device), lbls.to(device)
        opt.zero_grad()
        out = model(imgs)
        loss = criterion(out, lbls)
        loss.backward()
        opt.step()

        running_loss += loss.item()
        preds = out.argmax(dim=1)
        correct += (preds == lbls).sum().item()
        total += lbls.size(0)

    train_loss_hist.append(running_loss / len(train_loader))
    train_acc_hist.append(correct / total)

    # Validation
    model.eval()
    v_loss, v_correct, v_total = 0.0, 0, 0
    with torch.no_grad():
        for imgs, lbls in val_loader:
            imgs, lbls = imgs.to(device), lbls.to(device)
            out = model(imgs)
            loss = criterion(out, lbls)
            v_loss += loss.item()
            v_correct += (out.argmax(1) == lbls).sum().item()
            v_total += lbls.size(0)

    val_loss_hist.append(v_loss / len(val_loader))
    val_acc_hist.append(v_correct / v_total)

    sched.step()

    print(f"[{ep+1}/{epochs}] Train Acc: {train_acc_hist[-1]:.4f} | Val Acc: {val_acc_hist[-1]:.4f}")


[1/25] Train Acc: 0.8505 | Val Acc: 0.8839
[2/25] Train Acc: 0.9189 | Val Acc: 0.8908
[3/25] Train Acc: 0.9464 | Val Acc: 0.9041
[4/25] Train Acc: 0.9598 | Val Acc: 0.9079
[5/25] Train Acc: 0.9694 | Val Acc: 0.9071


In [ ]:
model.eval()
test_correct, test_total = 0, 0
pred_list, label_list = [], []

with torch.no_grad():
    for imgs, lbls in test_loader:
        imgs, lbls = imgs.to(device), lbls.to(device)
        out = model(imgs)
        pred = out.argmax(1)

        pred_list.extend(pred.cpu().numpy())
        label_list.extend(lbls.cpu().numpy())

        test_correct += (pred == lbls).sum().item()
        test_total += lbls.size(0)

test_acc = test_correct / test_total
print(f"\nTest Accuracy: {test_acc:.4f}")


In [ ]:
train_losses, val_losses = [], []
train_accuracies, val_accuracies = [], []


In [ ]:
train_losses = []
val_losses = []
train_accuracies = []
val_accuracies = []


In [ ]:
for epoch in range(epochs):
    model.train()
    correct, total, running_loss = 0, 0, 0.0

    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        _, preds = torch.max(outputs, 1)
        correct += (preds == labels).sum().item()
        total += labels.size(0)

    train_losses.append(running_loss / len(train_loader))
    train_accuracies.append(correct / total)

    model.eval()
    correct, total, running_loss = 0, 0, 0.0

    with torch.no_grad():
        for images, labels in val_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)

            running_loss += loss.item()
            _, preds = torch.max(outputs, 1)
            correct += (preds == labels).sum().item()
            total += labels.size(0)

    val_losses.append(running_loss / len(val_loader))
    val_accuracies.append(correct / total)

    scheduler.step()

    print(f"Epoch [{epoch+1}/{epochs}] | Train Acc: {train_accuracies[-1]:.4f} | Val Acc: {val_accuracies[-1]:.4f}")


Epoch [1/25] | Train Acc: 0.8485 | Val Acc: 0.8676
Epoch [2/25] | Train Acc: 0.9201 | Val Acc: 0.8922
Epoch [3/25] | Train Acc: 0.9457 | Val Acc: 0.8969


In [ ]:
model.eval()
correct, total = 0, 0
all_preds, all_labels = [], []

with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, preds = torch.max(outputs, 1)

        all_preds.extend(preds.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())

        correct += (preds == labels).sum().item()
        total += labels.size(0)

test_accuracy = correct / total
print(f"Final Test Accuracy: {test_accuracy:.4f}")


In [ ]:
plt.figure(figsize=(12,5))
plt.subplot(1,2,1)
plt.plot(train_losses, label="Train Loss")
plt.plot(val_losses, label="Val Loss")
plt.legend()

plt.subplot(1,2,2)
plt.plot(train_accuracies, label="Train Acc")
plt.plot(val_accuracies, label="Val Acc")
plt.legend()
plt.show()


In [ ]:
cm = confusion_matrix(all_labels, all_preds)
plt.figure(figsize=(8,6))
sns.heatmap(cm, annot=True, fmt="d", cmap="Blues")
plt.xlabel("Predicted")
plt.ylabel("True")
plt.show()


In [ ]:
class_names = ["Airplane","Automobile","Bird","Cat","Deer","Dog","Frog","Horse","Ship","Truck"]
print(classification_report(all_labels, all_preds, target_names=class_names))
